In [48]:
import json
import os
from typing import Any, Dict, List

import requests
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, ConfigDict, Field


In [4]:
load_dotenv()

GOOGLE_API_KEY = os.environ["GOOGLE_CUSTOM_SEARCH_API_KEY"]
GOOGLE_CX = os.environ["GOOGLE_CUSTOM_SEARCH_CX"]
GOOGLE_ENDPOINT = "https://www.googleapis.com/customsearch/v1"


In [40]:
def search_station_sign_images(station_name: str, count: int = 10) -> List[Dict[str, Any]]:
    """駅名を受け取り、駅名標っぽい画像候補を返す"""
    query = f"{station_name}駅 駅名標 写真"

    params = {
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CX,
        "searchType": "image",
        "q": query,
        "num": count,
        "safe": "active",
        "imgType": "photo",
    }

    res = requests.get(GOOGLE_ENDPOINT, params=params)
    res.raise_for_status()
    data = res.json()

    items = data.get("items", [])
    if not items:
        return []

    result = []
    for i, item in enumerate(items):
        image = item.get("image", None)
        if not image:
            continue

        result.append(
            {
                "id": f"img_{i + 1}",
                "snippet": item.get("snippet", ""),
                "link": item.get("link", ""),
                "thumbnailLink": image.get("thumbnailLink", ""),
                "width": image.get("width", 0),
                "height": image.get("height", 0),
                "rank": i + 1,
            }
        )

    return result

In [41]:
station_name = "横浜"
telop_text = "東京25分・新宿31分圏内"
content_text = "横浜駅からは東京駅へ約25分、新宿駅へ約31分、品川駅へ約12分と、主要ターミナルに乗換なしでアクセス可能。隣の桜木町駅へも約3分で、都心通いもしやすい立地です。"

In [42]:
search_result = search_station_sign_images("横浜")

In [43]:
search_result_json = "\n,".join(
    [json.dumps(sr, indent=2, ensure_ascii=False).replace("{", "{{").replace("}", "}}") for sr in search_result]
)

In [44]:
system_prompt = """
あなたは、サムネイル画像を選ぶアシスタントです。
与えられた画像候補（snippet / URL / width / height / rank）の
テキスト情報のみを使い、最適な1枚を選びます。

【目的】
「◯◯駅の駅名標」がシンプルに写っている写真を選び、
SNSサムネとして1秒で意味が伝わるものにします。


【評価ルール】
1. 駅名標らしさ  
   - snippet に「駅名標」「◯◯駅」などが含まれるほど高評価。
   - 路線図、広告、ポスター、観光案内が含まれる場合は減点。

2. 路線の一致
   - 駅名標の路線とテロップや内容文の路線が一致するほど高評価。

3. シンプルさ  
   - ごちゃごちゃした印象の語（路線図 / map / advertisement など）があれば減点。

4. 画像の大きさ  
   - 幅600px以上を優先。小さすぎる画像は減点。

5. 横長の構図  
   - width > height の場合はテロップを載せやすいため加点。

【テロップ位置推定】
- 横長なら "top_center"
- それ以外は "bottom_center" を推奨。

【出力】
以下の3つのみを JSON で返してください。文章は書かないこと。
- selected_id:（候補の id）
- recommended_text_position: "top_center" または "bottom_center"
- reason: 選んだ理由を簡潔に1〜2文で述べること。
"""

user_prompt = f"""
以下は、Google Custom Search JSON API（画像検索）で取得した
「{station_name}駅 駅名標 写真」に関する候補画像リストです。

この駅は、都内の主要ターミナル駅へのアクセスの良さを示すサムネイル用の「出発駅」です。

シーンの内容:
{content_text}

テロップ
{telop_text}

候補画像リストは JSON 配列で、各要素は次のフィールドを持ちます:
- id: こちらで付与した一意なID（例: "img_1"）
- snippet: 検索結果の説明文
- link: 画像URL
- thumbnailLink: サムネイル画像URL
- width: 画像の幅(px)
- height: 画像の高さ(px)
- rank: 検索結果の順位（1が最上位）。rankが小さいほど検索エンジンの評価が高いとみなせます。

候補画像リスト:
{search_result_json}

step by stepで考えてください。

1. まず、シーンの内容とテロップから、どの路線のアクセスの良さを示す必要があるかを特定してください。
2. 次に、各候補画像の snippet を分析し、駅名標らしさ、路線の一致、シンプルさ、画像の大きさ、横長の構図、検索順位に基づいて評価してください。
3. 最後に、最も評価の高い1枚を選び、その理由を簡潔に述べてください。
"""

In [50]:
# LLM用の分析ノードレスポンス
class StationSignResponse(BaseModel):
    """分析ノード用の構造化レスポンス"""

    model_config = ConfigDict(from_attributes=True)

    selected_id: str = Field(..., description="候補の id")
    reason: str = Field(..., description="選んだ理由を簡潔に1〜2文で述べること")

In [51]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = llm.with_structured_output(StationSignResponse)
thumbnail_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", user_prompt),
    ]
)

# 構造化LLMチェーンを実行
chain = thumbnail_prompt | llm
thumbnail_result = chain.invoke({})

In [52]:
thumbnail_result

StationSignResponse(selected_id='img_1', reason='この画像は「駅名標」が明確に写っており、横浜駅の駅名標としての認識が高いです。また、2048pxの幅があり、横長の構図でテロップを載せやすいため最適です。')

In [55]:
filtered_result = [sr for sr in search_result if sr["id"] == thumbnail_result.selected_id]

if len(filtered_result) > 0:
    selected_result = filtered_result[0]

selected_result

{'id': 'img_1',
 'snippet': 'File:京急本線横浜駅 駅名標.jpg - Wikimedia Commons',
 'link': 'https://upload.wikimedia.org/wikipedia/commons/b/b4/%E4%BA%AC%E6%80%A5%E6%9C%AC%E7%B7%9A%E6%A8%AA%E6%B5%9C%E9%A7%85_%E9%A7%85%E5%90%8D%E6%A8%99.jpg',
 'thumbnailLink': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQV03UF-a_7hGXDBc5N_BG4gGH2T2te3LD_RdUIkTbp90Iunwr3-aR8xyQ&s',
 'width': 2048,
 'height': 1536,
 'rank': 1}